In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import pandas as pd
from dont_patronize_me import DontPatronizeMe

In [4]:
dpm = DontPatronizeMe('.')
# This method loads the subtask 1 data
dpm.load_task1()
# which we can then access as a dataframe
dataset = dpm.train_task1_df
dataset.head()

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"we 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"in libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""white house press secretary sean spicer said ...",0,0
3,4,@@7811231,disabled,nz,council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" just like we received migrants fleeing el ...",0,0


In [5]:
dataset.label.value_counts()

0    9476
1     993
Name: label, dtype: int64

In [7]:
train = pd.read_csv("train_subset.csv")
valid = pd.read_csv("validation_subset.csv")

In [10]:
def text_preprocessing(text, lemmatize):
    if not isinstance(text, str):
        text = text.decode('ISO-8859-1')
    
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
                           "]+", flags = re.UNICODE)
    text = regrex_pattern.sub(r'',text)

    text = re.sub('[^a-zA-Z]', ' ', text) # Clear the special characters from our dataset
    text = text.lower() 
    text = text.split() 
    text = ' '.join(text)
    
    return text

train['cleaned'] = [text_preprocessing(doc, True) for doc in train.text]
valid['cleaned'] = [text_preprocessing(str(doc), True) for doc in valid.text]

In [11]:
y_train = train.label.to_numpy()
y_val = valid.label.to_numpy()

In [20]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights = dict(enumerate(class_weights))
class_weights

{0: 0.5523677615090358, 1: 5.273929471032746}

In [15]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], output_shape=[512,16], 
  dtype=tf.string,trainable= True)

INFO:absl:Using C:\Users\Raluca\AppData\Local\Temp\tfhub_modules to cache modules.


In [16]:
from tensorflow.keras.optimizers import Adam

In [17]:
def get_model(dense_1 = 128, dense_2 = 64):
  model = tf.keras.models.Sequential([
  hub_layer,
  tf.keras.layers.Dense(dense_1, activation='relu'),
  tf.keras.layers.Dense(dense_2, activation='relu'),
  tf.keras.layers.Dense(1)
  ])
  return model

In [21]:
from sklearn.metrics import f1_score, accuracy_score

model = get_model(int(122), int(32.44))

# Train the model for a specified number of epochs.
optimizer = Adam(learning_rate = 0.001014)
model.compile(loss = 'binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

model.fit(x = train.cleaned, y = y_train, epochs=7,
              batch_size=64, class_weight=class_weights, verbose=1)

Epoch 1/7
131/131 [==============================] - 287s 2s/step - loss: 0.6262 - accuracy: 0.7131
Epoch 2/7
131/131 [==============================] - 287s 2s/step - loss: 0.3938 - accuracy: 0.8907
Epoch 3/7
131/131 [==============================] - 285s 2s/step - loss: 0.1214 - accuracy: 0.9816
Epoch 4/7
131/131 [==============================] - 283s 2s/step - loss: 0.0599 - accuracy: 0.9963
Epoch 5/7
131/131 [==============================] - 281s 2s/step - loss: 0.0561 - accuracy: 0.9975
Epoch 6/7
131/131 [==============================] - 267s 2s/step - loss: 0.0560 - accuracy: 0.9975
Epoch 7/7
131/131 [==============================] - 263s 2s/step - loss: 0.0560 - accuracy: 0.9975


In [29]:
labels = ['REAL', 'FAKE']
prediction_labels=[]
for p in y_pred:
    prediction_labels.append(labels[np.argmax(p)])
    


['REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 'REAL',
 

In [25]:
y_pred = model.predict(valid.cleaned)
classes = np.argmax(y_pred, axis = 1)
print(classes)

[0 0 0 ... 0 0 0]


In [36]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, confusion_matrix

In [30]:
model.evaluate(x = valid.cleaned, y = y_val, steps=10, verbose=0)

[0.9157593846321106, 0.8839541673660278]

In [37]:
f1 = confusion_matrix(classes, y_val)
f1

array([[1895,  199],
       [   0,    0]], dtype=int64)

In [ ]:

def get_model(dense_1 = 128, dense_2 = 64):
  model = tf.keras.models.Sequential([
  hub_layer,
  tf.keras.layers.Dense(dense_1, activation='relu'),
  tf.keras.layers.Dense(dense_2, activation='relu'),
  tf.keras.layers.Dense(1)
  ])
  return model


def fit_with(verbose, dense_1, dense_2, lr):

    # Create the model using a specified hyperparameters.
    model = get_model(int(dense_1), int(dense_2))

    # Train the model for a specified number of epochs.
    optimizer = Adam(learning_rate = lr)
    model.compile(loss = 'binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    # Train the model with the train dataset.
    model.fit(x = train.cleaned, y = y_train, epochs=7,
              batch_size=64, verbose=verbose)

        # Evaluate the model with the eval dataset.
    score = model.evaluate(x = valid.cleaned, y = y_val, steps=10, verbose=0)
    print('Loss:', score[0])
    print('Accuracy:', score[1])

    # Return the mae.

    return score[1]

In [34]:
from functools import partial

verbose = 1
fit_with_partial = partial(fit_with, verbose)

from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'dense_1': (64, 256), 'dense_2': (16, 128), 'lr': (1e-4, 1e-2)}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=10, n_iter=10)


for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   |  dense_1  |  dense_2  |    lr     |
-------------------------------------------------------------
Epoch 1/7
131/131 [==============================] - 253s 2s/step - loss: 0.1877 - accuracy: 0.9057
Epoch 2/7
131/131 [==============================] - 251s 2s/step - loss: 0.0604 - accuracy: 0.9795
Epoch 3/7
131/131 [==============================] - 251s 2s/step - loss: 0.0161 - accuracy: 0.9983
Epoch 4/7
131/131 [==============================] - 251s 2s/step - loss: 0.0110 - accuracy: 0.9993
Epoch 5/7
131/131 [==============================] - 251s 2s/step - loss: 0.0098 - accuracy: 0.9994
Epoch 6/7
131/131 [==============================] - 251s 2s/step - loss: 0.0094 - accuracy: 0.9994
Epoch 7/7
131/131 [==============================] - 250s 2s/step - loss: 0.0093 - accuracy: 0.9994
Loss: 0.769253134727478
Accuracy: 0.8882521390914917
|  1        |  0.8883   |  144.1    |  96.68    |  0.000101 |
Epoch 1/7
131/131 [==============================] - 253s 2s/

Epoch 1/7
131/131 [==============================] - 254s 2s/step - loss: 0.3045 - accuracy: 0.9052
Epoch 2/7
131/131 [==============================] - 251s 2s/step - loss: 0.2687 - accuracy: 0.9052
Epoch 3/7
131/131 [==============================] - 251s 2s/step - loss: 0.2084 - accuracy: 0.9052
Epoch 4/7
131/131 [==============================] - 251s 2s/step - loss: 0.1536 - accuracy: 0.9052
Epoch 5/7
131/131 [==============================] - 251s 2s/step - loss: 0.1061 - accuracy: 0.9052
Epoch 6/7
131/131 [==============================] - 251s 2s/step - loss: 0.0708 - accuracy: 0.9543
Epoch 7/7
131/131 [==============================] - 251s 2s/step - loss: 0.0431 - accuracy: 0.9994
Loss: 0.6320094466209412
Accuracy: 0.8887296915054321
|  20       |  0.8887   |  124.3    |  55.48    |  0.0001   |
Iteration 0: 
	{'target': 0.8882521390914917, 'params': {'dense_1': 144.0682249028942, 'dense_2': 96.67634326552171, 'lr': 0.00010113231069171439}}
Iteration 1: 
	{'target': 0.90496659

In [6]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from dont_patronize_me import DontPatronizeMe
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
import re
import nltk
from nltk import word_tokenize
train = pd.read_csv("train_subset.csv")
valid = pd.read_csv("validation_subset.csv")

def text_preprocessing(text, lemmatize):
    if not isinstance(text, str):
        text = text.decode('ISO-8859-1')
    
    text = re.sub('[^a-zA-Z]', ' ', text) # Clear the special characters from our dataset
    text = text.lower() 
    text = text.split() 
    text = ' '.join(text)
    #print('\tClear Text.\n', text)

    # Tokenize
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #print('\tTokenizing.\n', tokens)

    # Removing the stopwords
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in set(stop)]
    #print('\tRemoving the stopwords.\n', tokens)
    
    # Remove words shorter than 3 characters
    tokens = [token for token in tokens if len(token) >= 2]
    #print('\tRemoving the words shorter than 3 characters\n', tokens)
    

    if lemmatize:
        # Lemmatize
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        #print('\tLemmatizing.\n', tokens)
    else:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
    
    # Preprocessed text
    tokens = ' '.join(tokens)
    
    return tokens

'''def text_preprocessing(text, lemmatize):
    if not isinstance(text, str):
        text = text.decode('ISO-8859-1')
    
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
                           "]+", flags = re.UNICODE)
    text = regrex_pattern.sub(r'',text)

    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower() 
    text = text.split() 
    text = ' '.join(text)
    
    return text'''

train['cleaned'] = [text_preprocessing(doc, True) for doc in train.text]
valid['cleaned'] = [text_preprocessing(str(doc), True) for doc in valid.text]

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer(min_df=1,
                                   max_df=0.7, # Removing frequent words like 'subject' that appear in every news
                                   ngram_range=(1,2), 
                                   lowercase=False, # we've already lowercased the text
                                   stop_words='english', 
                                   max_features=1340000)
X_train = vectorizer_tfidf.fit_transform(train.cleaned)
X_valid = vectorizer_tfidf.transform(valid.cleaned)

In [8]:
y_train = train.label.to_numpy()
y_val = valid.label.to_numpy()

In [12]:
from self_paced_ensemble import SelfPacedEnsembleClassifier
from self_paced_ensemble.self_paced_ensemble.base import sort_dict_by_key
from self_paced_ensemble.utils._plot import plot_2Dprojection_and_cardinality

from collections import Counter
import matplotlib.pyplot as plt

from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Train an SPE classifier
clf = SelfPacedEnsembleClassifier(
        base_estimator=DecisionTreeClassifier(), 
        n_estimators=10,
    ).fit(X_train, y_train)

# Predict with an SPE classifier
y_pred = clf.predict(X_valid)

In [14]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
print(f1_score(y_pred, y_val))
print(classification_report(y_pred, y_val))

0.2797814207650273
              precision    recall  f1-score   support

           0       0.69      0.95      0.80      1378
           1       0.64      0.18      0.28       716

    accuracy                           0.69      2094
   macro avg       0.67      0.56      0.54      2094
weighted avg       0.67      0.69      0.62      2094



In [15]:
from sklearn.ensemble import RandomForestClassifier
# Train an SPE classifier
clf = SelfPacedEnsembleClassifier(
        base_estimator=RandomForestClassifier(), 
        n_estimators=10,
    ).fit(X_train, y_train)

# Predict with an SPE classifier
y_pred = clf.predict(X_valid)
print(f1_score(y_pred, y_val))
print(classification_report(y_pred, y_val))

0.3120260021668472
              precision    recall  f1-score   support

           0       0.69      0.96      0.81      1370
           1       0.72      0.20      0.31       724

    accuracy                           0.70      2094
   macro avg       0.71      0.58      0.56      2094
weighted avg       0.70      0.70      0.63      2094



In [22]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [23]:
stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords,
    use_idf=True,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.75
    )

In [24]:
#Construct tfidf matrix and get relevant scores
tfidf = vectorizer.fit_transform(train.text).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [30]:
#Get POS tags for tweets and save as a string
tweet_tags = []
for t in train.text:
    tokens = basic_tokenize(preprocess(t))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    tag_str = " ".join(tag_list)
    tweet_tags.append(tag_str)

In [31]:
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
pos_vectorizer = TfidfVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None,
    use_idf=False,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.75,
    )
#Construct POS TF matrix and get vocab dict
pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}

In [32]:
#Now get other features
sentiment_analyzer = VS()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words)
    num_chars = sum(len(w) for w in words)
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet)
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [FKRA, FRE,syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)
other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "vader neg","vader pos","vader neu", \
                        "vader compound", "num_hashtags", "num_mentions", "num_urls", "is_retweet"]
feats = get_feature_array(train.text)

In [33]:
#Now join them all up
M = np.concatenate([tfidf,pos,feats],axis=1)
M.shape

(8375, 3933)

In [36]:
#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.items():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
for k,v in pos_vocab.items():
    pos_variables[v] = k

feature_names = variables+pos_variables+other_features_names

In [37]:
X = pd.DataFrame(M)
y = train.label.astype(int)

In [39]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l2",C=0.01))
X_ = select.fit_transform(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [40]:
model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr').fit(X_, y)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [41]:
y_preds = model.predict(X_)

In [43]:
print(f1_score(y_preds, y_train))

0.7142079477408819
